In [1]:
DATA_NAME = 'segmentAnything-full' 
TRANSFORM = 'wavelet-vertical'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_vertical_wavelet_full_segmentAnything.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,73031.44,-0.46,-0.03,0.30,0.01,-0.04,0.00,-0.00,-0.00
3,-0.46,18267.01,-0.11,0.05,-0.05,0.01,0.02,0.00,0.00
4,-0.03,-0.11,4479.32,0.07,0.00,0.00,-0.01,-0.00,0.00
5,0.30,0.05,0.07,1000.66,0.00,-0.00,0.00,0.00,0.00
6,0.01,-0.05,0.00,0.00,203.42,-0.00,-0.00,0.00,0.00
7,-0.04,0.01,0.00,-0.00,-0.00,37.83,0.00,0.00,0.00
8,0.00,0.02,-0.01,0.00,-0.00,0.00,6.40,0.00,0.00
9,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.99,-0.00
10,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.12


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00001,-0.00000,0.00004,0.00000,-0.00002,0.00001,-0.00001,-0.00000
3,-0.00001,1.00000,-0.00001,0.00001,-0.00003,0.00001,0.00004,0.00001,0.00000
4,-0.00000,-0.00001,1.00000,0.00004,0.00000,0.00000,-0.00003,-0.00000,0.00001
5,0.00004,0.00001,0.00004,1.00000,0.00000,-0.00002,0.00005,0.00000,0.00001
6,0.00000,-0.00003,0.00000,0.00000,1.00000,-0.00001,-0.00000,0.00002,0.00000
7,-0.00002,0.00001,0.00000,-0.00002,-0.00001,1.00000,0.00002,0.00000,0.00000
8,0.00001,0.00004,-0.00003,0.00005,-0.00000,0.00002,1.00000,0.00003,0.00000
9,-0.00001,0.00001,-0.00000,0.00000,0.00002,0.00000,0.00003,1.00000,-0.00000
10,-0.00000,0.00000,0.00001,0.00001,0.00000,0.00000,0.00000,-0.00000,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.8092096243820153

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[5.93275312e+08 3.65092800e+07 2.15006770e+06 1.04331780e+05
 4.14027493e+03 1.34333379e+02 3.42317966e+00 6.13811240e-02
 5.68547451e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,0.999415,-0.033286,-0.007619,-0.001691,-0.000344,-0.000065,-0.000011,-0.000002,-1.988442e-07
1,0.032963,0.998722,-0.037472,-0.007818,-0.001587,-0.000294,-0.000049,-0.000008,-8.949563e-07
2,0.008770,0.036852,0.998475,-0.039408,-0.007568,-0.001404,-0.000239,-0.000037,-4.275025e-06
3,0.002273,0.009122,0.038709,0.998267,-0.042636,-0.007554,-0.001271,-0.000197,-2.307485e-05
4,0.000554,0.002221,0.009059,0.041870,0.997876,-0.048374,-0.007839,-0.001206,-1.419805e-04
5,0.000129,0.000514,0.002099,0.009409,0.047453,0.997092,-0.058199,-0.008643,-1.015951e-03
6,0.000029,0.000115,0.000472,0.002106,0.010385,0.056878,0.995163,-0.078886,-8.957805e-03
7,0.000007,0.000026,0.000108,0.000484,0.002382,0.012818,0.076655,0.989974,-1.179246e-01
8,0.000002,0.000006,0.000026,0.000115,0.000567,0.003048,0.018020,0.116847,9.929816e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0005846487993962368,
 0.0012780295592710322,
 0.0015248054067814598,
 0.0017331554010748818,
 0.002123938138678616,
 0.00290816604225852,
 0.004837182196329892,
 0.010026474653247908,
 0.007018379995750945]